# 1 Multi-class Classification

In [1]:
import numpy as np
import scipy.io as sio

## 1.1 Dataset

In [2]:
data = sio.loadmat('data/ex3data1.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
# X is 5,000 examples of 20x20 pixel images, each image is a handwritten number
# Each 20x20 image is stored as a 400-dimensional vector
data['X'].shape

(5000, 400)

In [4]:
# y is the numeral represented by each image
data['y'].shape

(5000, 1)

In [5]:
# Store in numpy arrays to make things easier
X = data['X']
y = data['y']

## 1.2 Visualizing the data

In [6]:
m = X.shape[0] # number of examples

In [8]:
# Randomly select 100 images to display
rand_indices = np.random.permutation(m)
sel = X[rand_indices, :]

In [ ]:
def displayData(X, example_width):
    '''
    Displays 2D data stored in X in a nice grid.
    It returns the figure handle h and the displayed array if requested.
    '''
    
    
    
    return h, display_array